# Explaining and Interpreting Machine Learning Models

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a>&nbsp;| Dennis G. Wilson | <a href="https://github.com/d9w/interpretable_ml.git">https://github.com/d9w/interpretable_ml.git</a>

As machine learning models are used in more and more applications, explaining their functionality and their outputs becomes crucial. In critical applications like health and transportation, where an incorrect decision by an ML model can lead to serious consequences for people, understanding ML models is a must.

In this class, we'll look at understanding ML models used to predict heart disease.

There are multiple ways to understand a model, which are not mutually exclusive. We can **explain** data, the data features, the model's internals, and the model's predictions. Explainations use statistical measures to study different cases, ie, in 80% of cases, the model predicts X when a data feature has Y. We can also **interpret** the model, looking inside the model to see how it makes predictions.

<img src="https://d9w.github.io/interpretable_control/img/explaining.png">

Zhou, Ryan, and Ting Hu. "Evolutionary approaches to explainable machine learning." arXiv preprint arXiv:2306.14786 (2023).
https://arxiv.org/pdf/2306.14786

An explainable model is a function that is too complicated for a human to understand. Another name for this is a black-box model. We need an additional method/technique to be able to peer into the black-box and understand how the model works.

An interpretable model is one that can be understood by a human. However, as this is a subjective definition, the interpretability of a model is subject to opinion. One model is considered more interpretable than another if it is easier for a human to understand how it makes predictions than the other model.

<img src="https://miro.medium.com/v2/resize:fit:4800/format:webp/1*6-QWLr9obdDlUmVLtDXPrQ.png">

https://towardsdatascience.com/interperable-vs-explainable-machine-learning-1fa525e12f48

# Explaining Data

The first step to explaining data is to understand the source of the data. How was this information gathered? How is it organized? What are possible biases in the way in which it was gathered? 

The data we're looking at comes from phone surveys conducted by the [CDC](https://www.cdc.gov/brfss/annual_data/annual_2022.html) about people's health. The CDC website has more information on how this [Behavioral Risk Factor Surveillance System (BRFSS)](https://www.cdc.gov/brfss/annual_data/2022/pdf/Overview_2022-508.pdf) was conducted. We will look at predicting the `_MICHD` feature, which tracks if a surveyed person has had heart attack or diagnosed heart disease.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
np.seterr(divide='ignore', invalid='ignore');

In [ ]:
DATA_PATH = Path('data/')
RAW_DATA_PATH = DATA_PATH / 'raw'
RAW_FILE_PATH = RAW_DATA_PATH / 'LLCP2022.XPT'
PROCESSED_DATA_PATH = DATA_PATH / 'processed'
DATA_FILE = PROCESSED_DATA_PATH / 'data.csv'

In [ ]:
heart_df = pd.read_sas(RAW_FILE_PATH, encoding='utf-8')
heart_df.info()

In [ ]:
heart_df['ID'] = np.arange(len(heart_df))
heart_df.describe()

In [ ]:
var_list = {'ID': 'ID', 'SEXVAR': 'Sex', 'GENHLTH': 'GeneralHealth', '_STATE': 'State', 
            'PHYSHLTH': 'PhysicalHealth', 'MENTHLTH': 'MentalHealth', 'CHECKUP1': 'LastCheckup',
            'EXERANY2': 'Exercise', 'SLEPTIM1': 'Sleep', 'RMVTETH4': 'TeethRemoved', 'CVDSTRK3': 'Stroke',
            'ASTHMA3': 'Asthma', 'CHCSCNC1': 'SkinCancer', 'CHCCOPD3': 'LungDisease',
            'ADDEPEV3': 'Depression', 'CHCKDNY2': 'KidneyDisease', 'HAVARTH4': 'Arthritis',
            'DIABETE4': 'Diabetes', 'DEAF': 'Deaf', 'BLIND': 'Blind', 'DECIDE': 'DifficultyDeciding',
            'DIFFWALK': 'DifficultyWalking', 'DIFFDRES': 'DifficultyDressing', 'DIFFALON': 'DifficultyAlone',
            '_SMOKER3': 'Smoker', 'ECIGNOW2': 'ECigUser', '_AGEG5YR': 'Age', 'HTM4': 'Height',
            'WTKG3': 'Weight', '_BMI5': 'BMI', 'DRNKANY6': 'Alcohol', '_AIDTST4': 'HIVTest', 'FLUSHOT7': 'FluShot',
            'PNEUVAC4': 'Pneumonia', 'TETANUS1': 'Tetanus', 'HIVRISK5': 'HIVRisk', 'COVIDPOS': 'COVID',
            '_MICHD': 'HeartDisease'}

In [ ]:
df = heart_df[np.array(var_list.keys())]
dfpos = df[df['_MICHD'] == 1][:20000]
dfneg = df[df['_MICHD'] == 2][:20000]
df = pd.concat([dfpos, dfneg])
df = df.rename(columns=var_list)
df.describe()

In [ ]:
for c in df.columns:
    if df[c].max() == 99:
        df.loc[df[c] == 99, c] = -1
    if df[c].max() == 88:
        df.loc[df[c] == 88, c] = 0
    if df[c].max() == 77:
        df.loc[df[c] == 77, c] = -1
    if df[c].max() == 9:
        df.loc[df[c] == 9, c] = -1
    if df[c].max() == 7:
        df.loc[df[c] == 7, c] = -1
df = df.replace(-1, np.nan)
df = df.dropna()
df['HeartDisease'] = df['HeartDisease'].replace([1, 2], [1, 0])
df = df.sample(frac=1, random_state=1234)
df['ID'] = np.arange(len(df))
df.describe()

In [ ]:
df.to_csv(DATA_FILE, index=False)
#df = pd.read_csv(DATA_FILE)

## Visualization: General Health

In [ ]:
df_pie = df[['GeneralHealth', 'HeartDisease']].copy()
df_pie['GeneralHealth'] = df_pie['GeneralHealth'].replace([1, 2, 3, 4, 5],
                                ['Excellent', 'Very good', 'Good', 'Fair', 'Poor'])
grouped_data = df_pie.groupby(['GeneralHealth', 'HeartDisease']).size().unstack(fill_value=0)

# Converting the grouped data to proportions for the pie chart
pie_data = grouped_data.stack().reset_index(name='Count')
pie_data['Percentage'] = pie_data['Count'] / pie_data.groupby('GeneralHealth')['Count'].transform('sum') * 100

# Creating more concise labels
health_order = ['Excellent', 'Very good', 'Good', 'Fair', 'Poor']
pie_data['Label'] = pie_data.apply(lambda row: f"{row['GeneralHealth']} ({'+' if row['HeartDisease'] == 1 else '-'}) {row['Percentage']:.1f}%", axis=1)
pie_data['GeneralHealth'] = pd.Categorical(pie_data['GeneralHealth'], categories=health_order, ordered=True)

# Sorting the data
pie_data = pie_data.sort_values(by=['GeneralHealth', 'HeartDisease'], ascending=[True, True])

# Custom colors for Target == False and Target == True
colors_false = {'Excellent': '#98DF8A', 'Very good': '#FFFF99', 'Good': '#FFD92F',
                'Fair': '#FFBB78', 'Poor': '#FF9896'}
colors_true = {'Excellent': '#2CA02C', 'Very good': '#FFFF33', 'Good': '#E6AB02',
               'Fair': '#FF7F0E', 'Poor': '#D62728'}

# Applying the appropriate color based on Target
pie_data['Color'] = pie_data.apply(lambda row: colors_true[row['GeneralHealth']]
                                   if row['HeartDisease'] == 1
                                   else colors_false[row['GeneralHealth']], axis=1)

# Plotting the pie chart with the color scheme, exploded view
#explode_values = (0.05, 0.15, 0.05, 0.15, 0.05, 0.15, 0.05, 0.06, 0.05, 0.1)
explode_values = 0.05 * np.ones(len(pie_data['Label']))
# Creating custom legend
legend_elements = [mpatches.Patch(color='#E6AB02', label='Heart Disease is True when (+)'),
                   mpatches.Patch(color='#FFD92F', label='Heart Disease is False when (-)')]

plt.figure(figsize=(12, 12))
plt.pie(pie_data['Count'], labels=pie_data['Label'], autopct='%1.1f%%',
        startangle=90, colors=pie_data['Color'], explode=explode_values)
plt.title('Heart Disease Distribution by General Health', fontsize=16)
plt.legend(handles=legend_elements, loc='lower right')
plt.show();

## Visualization: Age and Sex

In [ ]:
# Prepare the data
grouped_data = df.groupby(['Sex', 'Age', 'HeartDisease']).size().unstack().reset_index()
grouped_data['Total'] = grouped_data[0] + grouped_data[1]
grouped_data['Sex'] = grouped_data['Sex'].replace([1, 2], ['Male', 'Female'])
grouped_data = grouped_data[grouped_data.Age != 14]
grouped_data['Age'] = grouped_data['Age'].replace([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
['Age 18 to 24', 'Age 25 to 29', 'Age 30 to 34', 'Age 35 to 39', 'Age 40 to 44', 'Age 45 to 49',
 'Age 50 to 54', 'Age 55 to 59', 'Age 60 to 64', 'Age 65 to 69', 'Age 70 to 74', 'Age 75 to 79', 'Age 80 or older'])

# Separate data for males and females
male_data = grouped_data[grouped_data['Sex'] == 'Male']
female_data = grouped_data[grouped_data['Sex'] == 'Female']

# Set the figure
fig, ax = plt.subplots(1, 1, figsize=(17, 8))
ax.barh(male_data['Age'], male_data[1], color='#1f77b4', alpha=0.8, label='Male: Heart Disease Yes')
ax.barh(female_data['Age'], -female_data[1], color='#ff7f0e', alpha=0.8, label='Female: Heart Disease Yes')

# Annotations and customization
for i in male_data['Age']:
    count = male_data[male_data['Age'] == i][1].values[0]
    ax.annotate(f"{count}", xy=(count, i), va='center', ha='right', fontweight='light', color='#4a4a4a',
                fontsize=10)
for i in female_data['Age']:
    count = female_data[female_data['Age'] == i][1].values[0]
    ax.annotate(f"{count}", xy=(-count, i), va='center', ha='left', fontweight='light', color='#4a4a4a',
                fontsize=10)

# Remove unnecessary axes and add titles
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_xticks([])
ax.legend().set_visible(False)
fig.text(0.16, 0.95, 'Heart Disease Distribution by Age and Sex',
         fontsize=15, fontweight='bold', fontfamily='serif')
fig.text(0.16, 0.90, 'Comparing the number of cases with heart disease across different age groups',
         fontsize=12, fontweight='light', fontfamily='serif')
fig.text(0.5, 0.85, "Male: Heart attacks", fontweight="bold",
         fontfamily='serif', fontsize=12, color='#1f77b4')
fig.text(0.3, 0.85, "Female: Heart attacks", fontweight="bold",
         fontfamily='serif', fontsize=12, color='#ff7f0e')

plt.show()

## Visualization: Sleep Hours

In [ ]:
# Grouping data by 'Sleep' and calculating the proportion of 'HeartDisease: True'
sleep_hours_grouped = df.groupby('Sleep')['HeartDisease'].value_counts(normalize=True).unstack()

# Calculating the proportion of 'HeartDisease: True' for each sleep hour
sleep_hours_proportion_true = sleep_hours_grouped[1]

sleep_hours_proportion_true
# Grouping data by 'Sleep' and calculating the proportion of 'HeartDisease: True'
sleep_hours_grouped = df.groupby('Sleep')['HeartDisease'].value_counts(normalize=True).unstack()


# Setting a more visually appealing style
sns.set(style="whitegrid")

# Enhancing the aesthetics of the bar chart

# Creating the plot
fig, ax = plt.subplots(figsize=(14, 8))
bars = ax.bar(sleep_hours_proportion_true.index, sleep_hours_proportion_true.values,
              color=sns.color_palette("twilight", len(sleep_hours_proportion_true)))

# Adding data labels
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

# Adjusting the grid and background
ax.set_facecolor('white')
ax.grid(color='grey', linestyle='-', linewidth=0.25, axis='y')

# Setting title and labels with customized font
ax.set_title('Proportion of Heart Disease Cases by Sleep Hours', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Sleep Hours', fontsize=12, fontweight='bold')
ax.set_ylabel('Proportion of Heart Disease', fontsize=12, fontweight='bold')

# Customizing ticks and limits
ax.set_xticks(sleep_hours_proportion_true.index)
ax.set_xticklabels(sleep_hours_proportion_true.index, rotation=45)
ax.set_ylim(0, max(sleep_hours_proportion_true.fillna(0)) + 0.05)  # Adding some space above the highest bar

plt.show()

# Explaining Machine Learning Model Predictions

We've seen how heart disease relates to certain features, but the goal of machine learning is to automate such analysis. Let's train an ML model and try to understand how it functions using statistical explanations.

In [ ]:
tdf = df.copy().drop('ID', axis=1)
tdf = (tdf-tdf.min())/(tdf.max()-tdf.min())
tdf = tdf.dropna()
X = tdf.copy().drop('HeartDisease', axis=1)
y = tdf['HeartDisease'].copy()
ntrain = int(len(X)*0.7)
train_x = X[:ntrain]
train_y = y[:ntrain]
test_x = X[ntrain:]
test_y = y[ntrain:];

In [ ]:
print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)
train_x.describe()

In [ ]:
# train an XGBoost model
import xgboost
model = xgboost.XGBClassifier().fit(train_x, train_y)
print("Train accuracy: ", model.score(train_x, train_y))
print("Test accuracy: ", model.score(test_x, test_y))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
sns.reset_orig() 
disp = ConfusionMatrixDisplay.from_estimator(model, test_x, test_y, normalize='all', cmap='pink')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(max_depth=3, n_estimators=30).fit(train_x, train_y)
print("Train accuracy: ", model_rf.score(train_x, train_y))
print("Test accuracy: ", model_rf.score(test_x, test_y))

In [ ]:
sns.reset_orig()
disp = ConfusionMatrixDisplay.from_estimator(model_rf, test_x, test_y, normalize='all', cmap='pink')

## SHAP

**SH**apley **A**dditive ex**P**lanations ([Lundberg et .al 2017](https://arxiv.org/abs/1905.04610))

In game theory, the [Shapley value](https://en.wikipedia.org/wiki/Shapley_value) (1953) is a measure for fairly distributing both gains and costs to several actors working in coalition.

The Shapley value applies primarily in situations when the contributions of each actor are unequal, but they work in cooperation with each other to obtain the payoff.

You first start by identifying each player’s contribution when they play individually, when 2 play together, and when all 3 play together. Their individual contributions could determine how important they are to they whole game, but the Shapley value is more fair. Which player do you think has the highest impact on the total payoff?

<p align="center">
<img src=https://clearcode.cc/wp-content/uploads/2016/11/ABC-wide.png?ver=1478561348 width=500>
</p>

Then, you need to consider all possible orders and calculate their marginal value – e.g. what value does each player add when player A enters the game first, followed by player B, and then player C.
Below are the 6 possible orders and the marginal value each player adds in the different combinations:
<p align="center">
<img src=https://clearcode.cc/wp-content/uploads/2016/11/ABC-updated.png?ver=1479258642 width=500>
</p>

Now that we have calculated each player’s marginal value across all 6 possible order combinations, we now need to add them up and work out the Shapley value (i.e. the average) for each player.

In [ ]:
a = [7, 7, 10, 3, 9, 10]
b = [4, 0, 4, 4, 4, 3]
c = [8, 12, 5, 12, 6, 6]
print("Check for sum: ", [a[i]+b[i]+c[i] for i in range(len(a))])
print("Shapley values:")
print("A: ", np.sum(a)/6)
print("B: ", np.sum(b)/6)
print("C: ", np.sum(c)/6)

Computing the Shapley value for each player will give the true contribution each player made to the game and assign credit fairly. In this case, player C has a higher contribution, even though their contribution when playing individually is smaller.

### SHAP for Explainability

To use SHAP for machine learning models, we will consider the features of our datas as players in the cooperative game of prediction. Each value of an independent variable or a feature for a given sample is a part of a cooperative game where we assume that prediction is actually the payout. Shapley values correspond to the contribution of each feature towards pushing the prediction away from the expected value.

In [ ]:
import shap

# explain the model's predictions using SHAP
explainer = shap.Explainer(model)
shap_values = explainer(train_x)

In [ ]:
shap_values.shape

In [ ]:
# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[1234, :])

The Shapley values are calculated for each row. As such, understanding the explanation overall of how the model functions is not very clear. Aggregate methods like beeswarm plots help with understanding the overall behavior of the model.

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
# explain the Random Forest model's predictions using SHAP
explainer = shap.Explainer(model_rf)
shap_values_rf = explainer(train_x)

In [ ]:
shap_values_rf.shape

A small note: due to how Random Forests output their predictions, the shape of the Shapley values is different. Each explanation is also split based on the label.

In [ ]:
shap.plots.waterfall(shap_values_rf[1234, :, 0])

In [ ]:
shap.plots.beeswarm(shap_values_rf[:,:,0])

SHAP is a state of the art method and has a great library. It is far from the only explanation method however, there others like [LIME](https://github.com/marcotcr/lime), [PDPs and ICE Plots](https://medium.com/towards-data-science/the-ultimate-guide-to-pdps-and-ice-plots-4182885662aa).

# Interpretable Machine Learning

However, what if we could go beyond explaining? Our model of a gradient boosting algorithm is a black-box, so we can't look inside of it. If we use an interpretable model, we can see how the decision is being made at every step of the model's logic. Decision Trees, the base component of Random Forests, are one such model that is considered interpretable.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

model = DecisionTreeClassifier(max_depth=4).fit(train_x, train_y)
print("Train accuracy: ", model.score(train_x, train_y))
print("Test accuracy: ", model.score(test_x, test_y))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
sns.reset_orig()
disp = ConfusionMatrixDisplay.from_estimator(model, test_x, test_y, normalize='all', cmap='pink')

In [ ]:
import graphviz
from sklearn import tree

dot_data = tree.export_graphviz(model, feature_names=train_x.columns,
                                filled=True, rounded=True, proportion=True,
                                precision=1, label='root')
graph = graphviz.Source(dot_data)  
graph 

Decision Trees illustrate also why interpretability is a spectrum. The larger the tree is, the harder it is to understand. Increase the max depth and replot the tree. Would you be able to explain this model to a doctor or a patient? Is it interpretable?

### Genetic Programming

Genetic Programming is a less well-known method for interpretable machine learning. Also known as symbolic regression, genetic programming combines predetermined functions in a representation like a tree, list, or graph. This graph is randomly modified and optimized using an evolutionary algorithm in order to improve model performance. A parameter known as parsimony determines if smaller, but slightly worse, models can be accepted. We will use the [gplearn](https://gplearn.readthedocs.io/) library, which implements tree-based GP in a scikit-learn syntax.

In [ ]:
from gplearn.genetic import SymbolicClassifier

model_gp = SymbolicClassifier(parsimony_coefficient=0.01,
                         feature_names=train_x.columns,
                         random_state=1)
model_gp.fit(train_x, train_y)
print("Train accuracy: ", model.score(train_x, train_y))
print("Test accuracy: ", model.score(test_x, test_y))

In [ ]:
disp = ConfusionMatrixDisplay.from_estimator(model_gp, test_x, test_y, normalize='all', cmap='pink')

In [ ]:
dot_data = model_gp._program.export_graphviz()
graph = graphviz.Source(dot_data)
graph.render('images/ex1_child', format='png', cleanup=True)
graph

In [ ]:
For now, the SHAP library isn't compatible with gplearn models, however, Shapley values can be determined for GP